# CSE 144 Group 3
## Variational Autoencoder

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
import random
import torch
from torch import nn, optim
import math
from IPython import display
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import pdb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(len(data))
data.head()

170653


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


# Data Preprocessing


In [ ]:
data["artists"] = data["artists"].str.replace("[", "")
data["artists"] = data["artists"].str.replace("]", "")
data["artists"] = data["artists"].str.replace("'", "")
data["artists"] = data["artists"].map(lambda row: row.split(', '))

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int64  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  float64
 5   duration_ms       170653 non-null  int64  
 6   energy            170653 non-null  float64
 7   explicit          170653 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  float64
 10  key               170653 non-null  int64  
 11  liveness          170653 non-null  float64
 12  loudness          170653 non-null  float64
 13  mode              170653 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  int64  
 16  release_date      17

In [ ]:
eleven_features = ['danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
                   'tempo', 'mode', 'valence', 'acousticness']
data = data[data.columns.intersection(eleven_features)]
data.head()

,valence,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo
0,0.0594,0.982,0.279,831667,0.211,0.878000,0.665,-20.096,1,0.0366,80.954
1,0.9630,0.732,0.819,180533,0.341,0.000000,0.160,-12.441,1,0.4150,60.936
2,0.0394,0.961,0.328,500062,0.166,0.913000,0.101,-14.850,1,0.0339,110.339
3,0.1650,0.967,0.275,210000,0.309,0.000028,0.381,-9.316,1,0.0354,100.109
4,0.2530,0.957,0.418,166693,0.193,0.000002,0.229,-10.096,1,0.0380,101.665


In [ ]:
x = np.array(data['tempo'])
y = np.array(data['loudness'])
scaler = MinMaxScaler()
data['tempo'] = scaler.fit_transform(x[:, np.newaxis])
data['loudness'] = scaler.fit_transform(y[:, np.newaxis])
data.head()

,valence,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo
0,0.0594,0.982,0.279,831667,0.211,0.878000,0.665,0.624916,1,0.0366,0.332450
1,0.9630,0.732,0.819,180533,0.341,0.000000,0.160,0.744797,1,0.4150,0.250243
2,0.0394,0.961,0.328,500062,0.166,0.913000,0.101,0.707071,1,0.0339,0.453125
3,0.1650,0.967,0.275,210000,0.309,0.000028,0.381,0.793736,1,0.0354,0.411113
4,0.2530,0.957,0.418,166693,0.193,0.000002,0.229,0.781521,1,0.0380,0.417503


In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(data, [int(0.7*len(data)), int(0.3*len(data))])
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [int(0.9*len(train_dataset)), int(0.1*len(train_dataset))])

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(11, 128)
        self.fc21 = nn.Linear(128, 150)
        self.fc22 = nn.Linear(128, 150)
        self.fc3 = nn.Linear(150, 128)
        self.fc4 = nn.Linear(128, 11)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc21(h), self.fc22(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def train_model():
  model = VAE()
  optimizer =
  for epoch in range(10):